In [1]:
import os

from joblib import Parallel, delayed
import numpy as np
import pandas as pd
import tensorflow._api.v2.compat.v1 as tf
import sklearn
from sklearn.model_selection import GroupKFold, LeaveOneGroupOut

from models import deeper_fcn as architecture
import algorithms.heartrate as hr
import utils

In [2]:
# tensorflow settings
tf.logging.set_verbosity(tf.logging.ERROR)
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
tf.keras.backend.set_session(tf.Session(config=config))

In [12]:
# load data
# x_data_train, y_data_train, groups_train = ...
# dummy:
train_size = 1000
n_groups = 28
data = np.genfromtxt('C:/Users/dario/Desktop/pr1/1000x400/datatrain.csv', delimiter=',')
# print the shape of the data
print(data.shape)
data = data[:len(data)-len(data)%400]
# reshape the data to be 2D array with 400 columns and 5 rows   (400, 5)
data = data.reshape( int(len(data)/400), 400)
# print the shape of the data
print(data.shape)
x_data_train = np.expand_dims(data, axis=2)
#y_train section
data = np.genfromtxt('C:/Users/dario/Desktop/pr1/1000x400/gold/gold.csv', delimiter=',')
print(len(data))
# compute average each 8 elements in the array
data = data[:len(data)-len(data)%16]
y_data_train = np.mean(data.reshape(int(len(data)/16), 16), axis=1)
print(y_data_train.shape)
# print first 2 rows of the data
print(y_data_train[:2])

groups_train = np.sort(np.random.randint(n_groups, size=train_size))

print(x_data_train.shape, y_data_train.shape, groups_train.shape)

#y_data_train = y_data_train.values
#x_data_train= x_data_train.values

'''
# read xtrain data from csv of bcg and store in numpy array
data = np.genfromtxt('data/mr1.csv', delimiter=',')
# print the shape of the data
print(data.shape)
data = data[:len(data)-len(data)%400]
# reshape the data to be 2D array with 400 columns and 5 rows   (400, 5)
data = data.reshape( int(len(data)/400), 400)
# print the shape of the data
print(data.shape)
x_train = np.expand_dims(data, axis=2)
data = np.genfromtxt('data/gold.csv', delimiter=',')
print(len(data))
# compute average each 8 elements in the array
data = data[:len(data)-len(data)%16]
y_train = np.mean(data.reshape(int(len(data)/16), 16), axis=1)
print(y_train.shape)
# print first 2 rows of the data
print(y_train[:2])'''

(400000,)
(1000, 400)
16000
(1000,)
[39.75  35.375]
(1000, 400, 1) (1000,) (1000,)


"\n# read xtrain data from csv of bcg and store in numpy array\ndata = np.genfromtxt('data/mr1.csv', delimiter=',')\n# print the shape of the data\nprint(data.shape)\ndata = data[:len(data)-len(data)%400]\n# reshape the data to be 2D array with 400 columns and 5 rows   (400, 5)\ndata = data.reshape( int(len(data)/400), 400)\n# print the shape of the data\nprint(data.shape)\nx_train = np.expand_dims(data, axis=2)\ndata = np.genfromtxt('data/gold.csv', delimiter=',')\nprint(len(data))\n# compute average each 8 elements in the array\ndata = data[:len(data)-len(data)%16]\ny_train = np.mean(data.reshape(int(len(data)/16), 16), axis=1)\nprint(y_train.shape)\n# print first 2 rows of the data\nprint(y_train[:2])"

In [13]:
enlarge = 1
model_params = dict(metrics=["mae", "mape"], enlarge=enlarge)
fit_params = dict(epochs=30, verbose=2)  # set epochs between 30 and 75

modelname = (architecture.__name__ + "-x{}".format(enlarge))
modelpath = os.path.join("output", modelname)
os.makedirs(os.path.join(modelpath, "final"), exist_ok=True)
print(modelpath)

# write model architecture to JSON file
model = architecture.create(**model_params)
with open(os.path.join(modelpath, "model.json"), "w") as fp:
    fp.write(model.to_json())

output\models.deeper_fcn-x1


In [14]:
# single process for parallel training
def process_split(xt, yt, i, fit_params):
    # set allow_growth in subprocess
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    tf.keras.backend.set_session(tf.Session(config=config))
    
    csv_path = os.path.join(modelpath, "logs-{:02d}.csv".format(i))
    weights_path = os.path.join(modelpath, "weights-{:02d}.h5".format(i))
    fit_params.update(callbacks=[tf.keras.callbacks.CSVLogger(csv_path)])
    
    model = architecture.create(**model_params)
    r = model.fit(xt, yt, **fit_params)
    
    model.save_weights(weights_path)
    tf.keras.backend.clear_session()

    return r.history

In [15]:
# split training data with LeaveOneGroupOut cross validation
splitter = utils.get_group_splitter(n_groups, groups_train)

In [16]:
rs = Parallel(n_jobs=4, verbose=10)(
    delayed(process_split)(x_data_train[t_inds], y_data_train[t_inds],
                           i, fit_params)
    for i, (t_inds, v_inds) in enumerate(splitter)
)

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   5 tasks      | elapsed:  5.1min
[Parallel(n_jobs=4)]: Done  10 tasks      | elapsed:  6.2min
[Parallel(n_jobs=4)]: Done  17 tasks      | elapsed:  8.3min
[Parallel(n_jobs=4)]: Done  24 out of  28 | elapsed:  9.7min remaining:  1.6min
[Parallel(n_jobs=4)]: Done  28 out of  28 | elapsed: 10.7min finished


In [19]:
model = utils.get_model_from_json(modelpath, "model.json")

# calculate MAPE and MAE for each left-out patient
splitter = LeaveOneGroupOut().split(x_data_train, y_data_train, groups=groups_train)
results = []
for i, (t_inds, v_inds) in enumerate(splitter):
    model.load_weights(os.path.join(modelpath, "weights-{:02d}.h5".format(i)))
    y_pred = model.predict(x_data_train[v_inds])
    y_true = y_data_train[v_inds]
    results.append((hr.hr_mape(y_true, y_pred), hr.hr_mae(y_true, y_pred)))
results = np.array(results)
display(results)

1/1 [==============================] - 0s 28ms/step


array([[-1, -1],
       [-1, -1],
       [-1, -1],
       [-1, -1],
       [-1, -1],
       [-1, -1],
       [-1, -1],
       [-1, -1],
       [-1, -1],
       [-1, -1],
       [-1, -1],
       [-1, -1],
       [-1, -1],
       [-1, -1],
       [-1, -1],
       [-1, -1],
       [-1, -1],
       [-1, -1],
       [-1, -1],
       [-1, -1],
       [-1, -1],
       [-1, -1],
       [-1, -1],
       [-1, -1],
       [-1, -1],
       [-1, -1],
       [-1, -1],
       [-1, -1]])

In [18]:
# train one model on entire training set

model = architecture.create(**model_params)
r = model.fit(x_data_train, y_data_train, **fit_params)
model.save_weights(os.path.join(modelpath, "final", "weights-00.h5"))
tf.keras.backend.clear_session()

Epoch 1/30
32/32 - 3s - loss: nan - mae: nan - mape: nan - 3s/epoch - 109ms/step
Epoch 2/30
32/32 - 1s - loss: nan - mae: nan - mape: nan - 747ms/epoch - 23ms/step
Epoch 3/30
32/32 - 1s - loss: nan - mae: nan - mape: nan - 717ms/epoch - 22ms/step
Epoch 4/30
32/32 - 1s - loss: nan - mae: nan - mape: nan - 720ms/epoch - 22ms/step
Epoch 5/30
32/32 - 1s - loss: nan - mae: nan - mape: nan - 718ms/epoch - 22ms/step
Epoch 6/30
32/32 - 1s - loss: nan - mae: nan - mape: nan - 720ms/epoch - 22ms/step
Epoch 7/30
32/32 - 1s - loss: nan - mae: nan - mape: nan - 707ms/epoch - 22ms/step
Epoch 8/30
32/32 - 1s - loss: nan - mae: nan - mape: nan - 711ms/epoch - 22ms/step
Epoch 9/30
32/32 - 1s - loss: nan - mae: nan - mape: nan - 702ms/epoch - 22ms/step
Epoch 10/30
32/32 - 1s - loss: nan - mae: nan - mape: nan - 806ms/epoch - 25ms/step
Epoch 11/30
32/32 - 1s - loss: nan - mae: nan - mape: nan - 944ms/epoch - 29ms/step
Epoch 12/30
32/32 - 1s - loss: nan - mae: nan - mape: nan - 957ms/epoch - 30ms/step
Epo